함수호출

In [1]:
import pandas as pd
import time,datetime
import numpy as np
from pandas import *
from informs import portfolio
import class_informs as inform

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import math
import random as ran
from scipy.stats import truncnorm
#import seaborn as sbn

import time,datetime
import cmath


import re
import matplotlib.pyplot as plt

데이터 전처리

In [2]:
#timeseries data
all_data=pd.read_csv('data/Timeseries_data_SP500.csv')

all_data['DATE']=pd.to_datetime(all_data['DATE'])
all_data=all_data.rename(columns={'NAME':'name','DATE':'date','SEDOL':'sedol','SECTOR':'sector','BETA':'beta','ALPHA_SCORE':'as','BENCH_WEIGHT':'bw',"MCAP_Q":'mq'})
sedol_list=all_data['sedol'].unique().tolist()

date_list=list(set(list(all_data['date'])))
date_list.sort()


dic_data = {k: v for k, v in all_data.groupby('date')}


results_=pd.read_csv('data/results_template.csv')
results_=results_.rename(columns={'DATE':'date','SEDOL':'sedol','WEIGHTS':'w','FOUR_WEEKLY_RETURN':'r'})
results_['date']=pd.to_datetime(results_['date'])
dic_results_={k: v for k, v in results_.groupby('date')}


def preprocessing(date):
    
    using_dic = dic_data[pd.to_datetime(date)]
    using_dic['index']=list(using_dic.index)

    asset_list = []
    for i in using_dic["sedol"]:
        asset_list.append(i)


    dic_sedol_as = {using_dic["sedol"][i] : using_dic["as"][i] for i in using_dic.index}

    dic_bench = {using_dic["sedol"][i] : using_dic["bw"][i] for i in using_dic.index}

    dic_beta = {using_dic["sedol"][i] : using_dic["beta"][i] for i in using_dic.index}

    dic_sector = {using_dic["sector"][i] : [] for i in using_dic.index }

    dic_MCAP = {using_dic["mq"][i] : [] for i in using_dic.index}

    for i in using_dic.index:
        dic_sector[using_dic["sector"][i]].append(using_dic["sedol"][i])
        dic_MCAP[using_dic["mq"][i]].append(using_dic["sedol"][i])



    #risk:cov_mat
    date_str=str(date)[:10]
    risk_data=pd.read_csv('data/Riskmodels/cov_mat_%s.csv'%(date_str))
    risk_sedol=risk_data['ROW_INDEX'].unique().tolist()
    risk_mat = np.zeros((len(risk_sedol),len(risk_sedol)))
    risk_mat[np.triu_indices(len(risk_sedol), 0)] = list(risk_data['VALUE'])
    irows,icols = np.triu_indices(len(risk_sedol),0)
    risk_mat[icols,irows]=risk_mat[irows,icols]
    #         return risk_data,risk_sedol,risk_mat

    
    alpha = []


    for i in risk_sedol:

        alpha.append(-1*dic_sedol_as[i])

    dic_r={dic_results_[date]["sedol"][i] : dic_results_[date]["r"][i] for i in using_dic.index}

    return using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha

print("complete")

complete


함수 정의

In [11]:
def cal_obj1(sol, risk_mat, as_) :

    a = np.full((len(risk_mat[0]), len(risk_mat[0])), 10000)
    risk_mat = risk_mat * a
    s_rr = np.dot(sol, risk_mat)
    s_r = np.dot(s_rr, sol)
    s_a = np.dot(sol, as_*10000)
    obj = s_r-s_a
    return obj

def cal_obj(sol, risk_mat, as_) :

    a = np.full((len(risk_mat[0]), len(risk_mat[0])), 10000)
    risk_mat = risk_mat * a
    sol_d = list(np.array(sol) - np.array(using_dic['bw']))
    s_rr = np.dot(sol_d, risk_mat)
    s_r = np.dot(s_rr, sol_d)
    s_a = np.dot(sol_d, as_*10000)
    obj = s_r-s_a
    return obj



def select_candi(sol) :
    
    diff_sub = np.array([0]*len(as_), float)
    diff_add = np.array([0]*len(as_), float)
    candi = []
    
#     if sum(sol) >= 50 :
#         for val in range(len(sol)) :
#             if sol[val] > 0. :
#                 candi.append(val)
        
#     else :
    for i in range(len(sol)) :
        if(sol[i] > 0) :
            w = sol[i]
            sol[i] = 0
            diff_sub[i] = cal_obj(sol, risk_mat, as_)
            if min(diff_sub[diff_sub>0]) == diff_sub[i] :
                for j in range(len(sol)) :
                    if sol[j] == 0 :
                        sol[j] = w
                        diff_add[j] = cal_obj(sol, risk_mat, as_)
                        sol[j] = 0
            sol[i] = w
        
    sol[np.where(diff_sub == min(diff_sub[diff_sub>0]))[0][0]] = 0
    sol[np.where(diff_add == min(diff_add[diff_add>0]))[0][0]] = 0.1

    for val in range(len(sol)) :
        if sol[val] > 0. :
            candi.append(val)
    print(candi)
    print("candi")

    return candi



def update(sol,lim_time,as_,w_pre,O_scale) :
    start_time = time.time()
    
    
    
    
    ex = inform.informs(10000, 10000, w_pre, O_scale,dic_sector,dic_bench,risk_sedol,dic_MCAP,dic_beta,alpha,risk_mat)
    ex.set_omega(risk_mat)

    new_list = []

    con = []
    nums = 0
    for k in risk_sedol :
        new_list.append(sol[k])
        if sol[k] == 1:
            con.append(nums)
        nums += 1
    result_list = []
    result_list2 = []
    
    
    
    print(con)
    print("con")
    while(True):

        if time.time() - start_time > lim_time:
            break
        
        ex.set_con(cons=con)
        new_list2 = []
        new_list3 = []
        try:
            aabbaa = ex.solve(0.01)
            
            
            if aabbaa ==1234:
                break

            if aabbaa != 0:
                aaaa = aabbaa[0]
                bbbb= aabbaa[1]
                for k in risk_sedol :
                    new_list2.append(aaaa[k])
                for k in risk_sedol :
                    new_list3.append(bbbb[k])
                new_list = new_list2

                result_list.append(aabbaa[2])
                result_list2.append(aabbaa[0])
                print("obj_val")
                print(cal_obj1(new_list3, risk_mat, as_))
                con = select_candi(new_list)
                print("cplex_val")
                print(aabbaa[2])

            else:
                print("TE infeasible")
                con = select_candi(new_list)

        except:
            print("cplex no solution")
            con = select_candi(new_list)



    return result_list,result_list2

def update2(sol,lim_time,as_,w_pre,O_scale) :
    start_time = time.time()
    
    
    
    
    ex = inform.informs(10000, 10000, w_pre, O_scale,dic_sector,dic_bench,risk_sedol,dic_MCAP,dic_beta,alpha,risk_mat)
    ex.set_omega(risk_mat)

    new_list = []

    con = []
    nums = 0
    for k in risk_sedol :
        new_list.append(sol[k])
        if sol[k] == 1:
            con.append(nums)
        nums += 1
    result_list = []
    result_list2 = []
    
    
    
    print(con)
    print("con")
    while(True):

        if time.time() - start_time > lim_time:
            break
        
        ex.set_con(cons=con)
        new_list2 = []
        new_list3 = []

        aabbaa = ex.solve(0.01)


        if aabbaa != 0:
            aaaa = aabbaa[0]
            bbbb= aabbaa[1]
            for k in risk_sedol :
                new_list2.append(aaaa[k])
            for k in risk_sedol :
                new_list3.append(bbbb[k])
            new_list = new_list2

            result_list.append(aabbaa[2])
            result_list2.append(aabbaa[0])
            con = select_candi(new_list)


        else:
            print("TE infeasible")
            con = select_candi(new_list)





    return result_list,result_list2


def sample_Z(m, n):
    return Variable(torch.Tensor(np.random.uniform(0., 1., size=[m, n])))

        
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / np.sqrt(in_dim / 2.)
    return np.random.normal(size=size, scale=xavier_stddev)


class Generator(nn.Module):
    def __init__(self,  in_dim=10, hd_dim=50, out_dim=500):
        super(Generator, self).__init__()
        
        self.G_W1 = nn.Parameter(torch.from_numpy(xavier_init([in_dim, hd_dim])).float())
        self.G_b1 = nn.Parameter(torch.from_numpy(np.zeros(shape=[hd_dim])).float())
        
    
        self.G_W12 = nn.Parameter(torch.from_numpy(xavier_init([hd_dim, hd_dim])).float())
        self.G_b12 = nn.Parameter(torch.from_numpy(np.zeros(shape=[hd_dim])).float())
        
        self.G_W13 = nn.Parameter(torch.from_numpy(xavier_init([hd_dim, hd_dim])).float())
        self.G_b13 = nn.Parameter(torch.from_numpy(np.zeros(shape=[hd_dim])).float())

        
        self.G_W2 = nn.Parameter(torch.from_numpy(xavier_init([hd_dim, out_dim])).float())
        self.G_b2 = nn.Parameter(torch.from_numpy(np.zeros(shape=[out_dim])).float())

    def forward(self, x):
        x = F.relu( x @ self.G_W1 + self.G_b1)
        x = F.relu( x @ self.G_W12 + self.G_b12)
        #x = F.relu( x @ self.G_W13 + self.G_b13)
        x = x @ self.G_W2 + self.G_b2

        return F.sigmoid(x*0.1)


class Discriminator(nn.Module):
    def __init__(self, in_dim=500,train=False):
        super(Discriminator, self).__init__()
        self.tot_infs = 0.0
        self.tot_infs_nor = 0.0
        self.train = train
        self.x_threshold = 0.001
        self.inf_scale = 100.0
        self.Zstar = 1000
        
        self.in_dim = in_dim

        np.random.seed(0)
        self.Omega = Variable(torch.from_numpy(risk_mat)).float()

        self.card_upper = 70.
        self.card_lower = 65.
        
        
    def forward(self, x):
        tot_infs = Variable(torch.zeros(x.size()[0]))
        x_bw = Variable(torch.from_numpy(bw).float())
        alp = Variable(torch.Tensor(alpha).float())   
        bet = Variable(torch.Tensor(beta).float())
 
        # (4)
        tot_infs += F.relu(1. - torch.sum(x,1))
        tot_infs += F.relu(torch.sum(x,1) - 1.)
        
        # (5)
        tot_infs += F.relu(torch.abs(x-x_bw)@ Variable(torch.ones(self.in_dim))-0.05)

        # (6)
        for j in range(nbsector):
            k= (x-x_bw) @ Variable(
                torch.Tensor([1 if i in group_by_sector[j] else 0 for i in range(self.in_dim)]).float())  #sum
            l = torch.abs(k)
            tot_infs += F.relu(l - 0.1)
            
        # (7)
        for j in range(nbmq):
            k = (x-x_bw) @ Variable(
                torch.Tensor([1 if i in group_by_mq[j] else 0 for i in range(self.in_dim)]).float())  #sum
            l = torch.abs(k)
            tot_infs += F.relu(l - 0.1)
            
        # (8)
        k = (x - x_bw)@bet
        l = torch.abs(k)
        tot_infs += F.relu(l - 0.1)
        
        # (9) lb <= card(x) <= b
        if self.train:
            num_non_zeros = torch.sum(torch.tanh(x / self.x_threshold), 1)
            tot_infs += F.relu(num_non_zeros - self.card_upper) + F.relu(self.card_lower - num_non_zeros)
        else:
            num_non_zeros = torch.sum(x.data>self.x_threshold, 1).float()
            tot_infs += Variable(np.maximum(num_non_zeros - self.card_upper, 0) + np.maximum(self.card_lower - num_non_zeros, 0))
    
        
        # (10)
        l = 0.5*torch.sum(torch.abs(x - x_bw))
        tot_infs += F.relu(0.6 - l)
        #tot_infs += F.relu(l - 1.)
    
        
        # (11) a<=dQd <= b
        dQ = (x-x_bw) @ self.Omega
        size_x = x.size()
        dQd = torch.bmm(dQ.view(size_x[0],1,size_x[1]), (x-x_bw).view(size_x[0],size_x[1],1)).squeeze(1).squeeze(1)
        tot_infs += F.relu(dQd - 0.01)
        tot_infs += F.relu(0.0025 - dQd)
        
        
        # min dQd + ad (<= Zstar)
        l = (x-x_bw)@alp

        MONTH =0
        turnover=x_bw
        
        if MONTH > 0 :
            tot_infs += (F.relu(100.*dQd - 100.*l -0.5*turnover - self.Zstar))
            self.objs = 100.*dQd - 100.*l -0.5*turnover
        else:
            tot_infs += (F.relu(100.*dQd - 100.*l  - self.Zstar ))
            self.objs = 100.*dQd - 100.*l
        
        
        fea_probs = F.relu(1. - F.tanh(tot_infs/self.inf_scale))  

        
        return fea_probs
        


def GAN(rotation):
    X_dim = nbasset
    HL_dim = 50
    Z_dim = 10

    G = Generator(Z_dim, HL_dim, X_dim)

    sample_D = Discriminator(X_dim, train=False)
    train_D = Discriminator(X_dim, train=True)

    G_solver = optim.Adam(G.parameters(), lr=1e-3)

    mb_size = 128

    best_obj = 100000.0
    best_sols = []

    losses = []

    start_time = time.time()

    feasible_threshold = 0.95

    adaptive_inf_scale = float(nbasset)

    for it in range(rotation):
        if it % 900 == 0:
            sample_noise = sample_Z(128, Z_dim)
            samples = G(sample_noise)

            sample_D.Zstar = best_obj
            sample_D.inf_scale = adaptive_inf_scale

            sample_D_values = sample_D(samples).data.numpy()
            sample_obj_values = sample_D.objs.data.numpy()

            fea_sample_obj_values = sample_obj_values[sample_D_values > feasible_threshold]
            sols = samples.data.numpy()[sample_D_values > feasible_threshold]

            if len(fea_sample_obj_values > 0):
                best_idx = np.argmin(fea_sample_obj_values)
                if np.min(fea_sample_obj_values) < best_obj:
                    best_obj = np.minimum(fea_sample_obj_values[best_idx], best_obj)
                    best_sols.append(sols[best_idx])

        z = Variable(torch.randn(mb_size, Z_dim))

        train_D.Zstar = best_obj
        train_D.inf_scale = adaptive_inf_scale

        G_loss = -torch.mean(torch.log(train_D(G(z))))

        G.zero_grad()
        G_loss.backward()
        G_solver.step()
        
    sam=[]
    for i in range(50):
        n = sample_Z(128, 10)
        samples = G(n)
        sam.append(samples)
        
    return best_sols, sols,fea_sample_obj_values,sam

        

def GAN_reulst(best_sols, sols,fea_sample_obj_values):    
    good_sols=np.argsort(fea_sample_obj_values)[:10]
    card_list=[list(np.where(sols[i]>0.001)[0]) for i in range(len(good_sols))]
    card_freq=np.zeros(nbasset)
    for i in range(len(card_list)):
        for j in card_list[i]:
            card_freq[j]+=1
    best=np.argsort(sum(sols))[::-1][:len(card_freq[card_freq>len(card_list)-5])]
    bestsol=np.where(best_sols[0]>0.001)[0]
    return best,bestsol

def init_cplex(risk_mat,risk_sedol):
    sedol_var_list =[]
    for i in risk_sedol:
        sedol_var_list.append("d"+str(i))

    for i in risk_sedol:
        sedol_var_list.append("q"+str(i))

    sedol_var_list.append("assum")
    #global alpha
    alpha = []
    for i in risk_sedol:
        alpha.append(-10000.*dic_sedol_as[i])

    qmat=[]
    for i in range(len(risk_mat)):
        qmat_1=[]
        qmat_1.append(sedol_var_list)
        new_risk_mat=[]
        for j in risk_mat[i]:
            new_risk_mat.append(20000*j)
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        new_risk_mat.append(0)
        qmat_1.append(new_risk_mat)
        qmat.append(qmat_1)

    for i in range(len(risk_mat)):
        qmat_1=[]
        qmat_1.append(sedol_var_list)
        new_risk_mat=[]
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        new_risk_mat.append(0)
        qmat_1.append(new_risk_mat)
        qmat.append(qmat_1)

    for i in range(1):
        qmat_1=[]
        qmat_1.append(sedol_var_list)
        new_risk_mat=[]
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        new_risk_mat.append(0)
        qmat_1.append(new_risk_mat)
        qmat.append(qmat_1)

    q_con1 = []
    q_con2 = []
    q_val = []


    for i in range(len((risk_mat[0]))):
        for j in range(len(risk_mat[0])):
            if j >= i:
                q_con1.append(i)
                q_con2.append(j)
                if i == j:
                    ex_list = list(risk_mat[i])
                    q_val.append(0.5*ex_list[j]*10000)
                else:
                    ex_list = list(risk_mat[i])
                    q_val.append(ex_list[j]*10000)

    Q_con = []
    Q_con.append(q_con1)
    Q_con.append(q_con2)
    Q_con.append(q_val)
    
    return Q_con, qmat, alpha

print("complete")

complete


solve!

In [12]:
#for i in date: 이런식으로 하되 evaluation criteria 는 아직 반영안함..!
feasi = 0
infeasi = 0
list_feasi_obj = []


############################################
for k in range(1):
    print(k, "번째")
    n=0
    date=date_list[n]
    init_set='GAN' #initial set by GAN, cplex, random

    c_time = 10. #cplex time for initial set
    gan_rot = 1800 #GAN epoch size for initial set
############################################


#preprocessing
    using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha=preprocessing(date)


#set initial cardinality
    if init_set=='cplex':
        Q_con, qmat, alpha = init_cplex(risk_mat,risk_sedol)
        sol = portfolio(sector=dic_sector,bench=dic_bench,asset=risk_sedol,
                MCAPQ=dic_MCAP,beta=dic_beta,alpha=alpha,qmat=qmat, Q_con=Q_con,multiple=10000,time_init=90)[3][0]
        alpha = []


        for i in risk_sedol:

            alpha.append(-1*dic_sedol_as[i])
    elif init_set=='aa':
        print("a")
    elif init_set=='GAN':

        bw=list(using_dic['bw'])
        bw=np.array(bw)

        beta=list(using_dic['beta'])
        beta=np.array(beta)

        alpha=list(using_dic['as'])
        alpha=np.array(alpha)

        sector_set = set(list(using_dic['sector']))
        sector_list = list(using_dic['sector'])

        gbs=using_dic[['sector', 'index']].apply(tuple, axis=1)
        group_by_sector = [[y[1] for y in gbs if y[0]==x] for x in sector_set]

        mq_list=list(using_dic['mq'])
        mq_set=set(list(using_dic['mq']))

        gbm=using_dic[['mq', 'index']].apply(tuple, axis=1)
        group_by_mq = [[y[1] for y in gbm if y[0]==x] for x in mq_set]

        nbasset=len(bw)
        nbsector=len(sector_set)
        nbmq=len(mq_set)

        best_sols, sols,fea_sample_obj_values,samples=GAN(gan_rot)
       # best,bestsol=GAN_reulst(best_sols, sols,fea_sample_obj_values)

        #print(len(best))


        aa=np.zeros(nbasset)
        for i in range(50):
            sb =samples[i].data.numpy()
            bb=sum(sb>0.0001)
            aa=aa+bb
        general=np.where(aa.max()*0.001<aa)
        good=np.where(aa.max()*0.1<aa)
        plus=list(set(general[0])-set(good[0]))



        best=list(good[0])
        sol={}
        for i in range(len(risk_sedol)):
            if i in best: sol[risk_sedol[i]]=1.
            else: sol[risk_sedol[i]]=0.


        best1=list(good[0])+list(np.random.choice(plus, np.random.randint(3,5)))
        sol1={}
        for i in range(len(risk_sedol)):
            if i in best1: sol1[risk_sedol[i]]=1.
            else: sol1[risk_sedol[i]]=0.

        best2=list(good[0])+list(np.random.choice(plus, np.random.randint(3,5)))        
        sol2={}
        for i in range(len(risk_sedol)):
            if i in best2: sol2[risk_sedol[i]]=1.
            else: sol2[risk_sedol[i]]=0.

        best3=list(good[0])+list(np.random.choice(plus, np.random.randint(3,5)))        
        sol3={}
        for i in range(len(risk_sedol)):
            if i in best3: sol3[risk_sedol[i]]=1.
            else: sol3[risk_sedol[i]]=0.

        best4=list(good[0])+list(np.random.choice(plus, np.random.randint(3,5)))        
        sol4={}
        for i in range(len(risk_sedol)):
            if i in best4: sol4[risk_sedol[i]]=1.
            else: sol4[risk_sedol[i]]=0.

        best5=list(good[0])+list(np.random.choice(plus, np.random.randint(3,5)))        
        sol5={}
        for i in range(len(risk_sedol)):
            if i in best5: sol5[risk_sedol[i]]=1.
            else: sol5[risk_sedol[i]]=0.       

        best6=list(good[0])+list(np.random.choice(plus, np.random.randint(3,5)))                
        sol6={}
        for i in range(len(risk_sedol)):
            if i in best6: sol6[risk_sedol[i]]=1.
            else: sol6[risk_sedol[i]]=0.       

        best7=list(good[0])+list(np.random.choice(plus, np.random.randint(3,5)))                
        sol7={}
        for i in range(len(risk_sedol)):
            if i in best7: sol7[risk_sedol[i]]=1.
            else: sol7[risk_sedol[i]]=0.

    elif init_set=='random':
        r=list(set(list(np.random.randint(len(risk_sedol),size=70))))
        sol={}
        for i in range(len(risk_sedol)):
            if i in r: sol[risk_sedol[i]]=1.
            else: sol[risk_sedol[i]]=0.


# solve & update


    as_ = using_dic['as']
    if n == 0:
        w_pre = {}
        for j in risk_sedol:
            w_pre.update({j:0})
        results = update(sol,80.,as_,w_pre,0)
            
    else:
        results = update(sol,80.,as_,w_pre,1)

    result = results[0]
    try:
        w_index = result.index(min(result))
        result2 = results[1][w_index]
        print(result2)

        w_pre = w_i_pre_sum(result2)
        feasi += 1
        list_feasi_obj.append(result)
        print("------objective value 변화------")
        print(result)
        
    except:
        infeasi += 1

        
print("feasible 횟수 : ",feasi)
print("저장된 objective value")
print(list_feasi_obj)
print("infeasible 횟수 : ",infeasi)

0 번째
[4, 9, 19, 20, 48, 54, 55, 82, 90, 94, 95, 98, 100, 106, 109, 111, 122, 124, 126, 133, 137, 139, 140, 142, 143, 147, 148, 151, 156, 157, 163, 166, 167, 171, 176, 178, 190, 196, 198, 199, 200, 201, 202, 212, 214, 215, 218, 221, 223, 229, 234, 235, 240, 241, 246, 247, 248, 253, 254, 255, 256, 257, 260, 268, 270, 274, 275, 276, 280, 283, 287, 290, 291, 292, 295, 303, 312, 313, 317, 326, 339, 342, 351, 353, 355, 365, 373, 374, 376, 377, 379, 390, 395, 401, 406, 408, 414, 416, 418, 421, 422, 423, 425, 426, 428, 433, 434, 437, 444, 446, 452, 454, 455, 456, 460, 470, 483, 486, 490]
con
0
AC
0.791145579347
TE
[ 15.68654269]
1
TE :  [ 32.51991078]
AS :  0.803848967923
2
TE :  [ 35.43872244]
AS :  0.802507686664
Feasible
cplex no solution
[4, 9, 19, 20, 48, 54, 55, 82, 90, 94, 95, 98, 100, 106, 109, 111, 122, 124, 126, 133, 137, 139, 140, 142, 143, 147, 148, 151, 156, 157, 163, 166, 167, 171, 176, 178, 190, 196, 198, 199, 200, 201, 202, 212, 214, 215, 218, 221, 223, 229, 234, 235, 240, 241,

19
AC
0.77050532533
TE
[ 13.30438739]
20
TE :  [ 29.70371268]
AS :  0.770152927098
Feasible
cplex no solution
[1, 4, 11, 13, 17, 19, 20, 21, 26, 48, 53, 54, 55, 58, 66, 70, 72, 82, 90, 94, 95, 98, 100, 106, 109, 111, 122, 124, 133, 137, 139, 140, 142, 143, 147, 148, 151, 156, 157, 163, 166, 167, 171, 176, 190, 196, 198, 199, 201, 202, 212, 214, 215, 218, 221, 228, 229, 234, 235, 240, 241, 246, 247, 248, 253, 254, 255, 256, 257, 260, 270, 274, 275, 276, 277, 280, 283, 287, 290, 291, 292, 295, 312, 313, 317, 326, 339, 351, 353, 373, 374, 376, 377, 379, 390, 395, 406, 408, 416, 418, 421, 422, 423, 425, 426, 428, 433, 434, 437, 444, 446, 452, 454, 455, 456, 460, 483, 486, 490]
candi
21
AC
0.769670233571
TE
[ 29.72575569]
infeasible
TE infeasible
[1, 4, 11, 13, 17, 19, 20, 21, 26, 42, 48, 53, 54, 55, 58, 66, 70, 72, 82, 90, 94, 95, 98, 100, 106, 109, 111, 122, 124, 133, 137, 139, 140, 142, 143, 147, 148, 151, 156, 157, 163, 166, 167, 171, 176, 190, 196, 198, 199, 201, 202, 212, 214, 215, 21

In [14]:


# solve & update

sol=sol2
as_ = using_dic['as']
if n == 0:
    w_pre = {}
    for j in risk_sedol:
        w_pre.update({j:0})
    results = update(sol,80.,as_,w_pre,0)

else:
    results = update(sol,80.,as_,w_pre,1)

result = results[0]
try:
    w_index = result.index(min(result))
    result2 = results[1][w_index]
    print(result2)

    w_pre = w_i_pre_sum(result2)
    feasi += 1
    list_feasi_obj.append(result)
    print("------objective value 변화------")
    print(result)

except:
    infeasi += 1

        
print("feasible 횟수 : ",feasi)
print("저장된 objective value")
print(list_feasi_obj)
print("infeasible 횟수 : ",infeasi)

[4, 9, 19, 20, 48, 54, 55, 82, 83, 90, 94, 95, 98, 100, 106, 109, 111, 115, 122, 124, 126, 133, 137, 139, 140, 142, 143, 147, 148, 151, 156, 157, 163, 166, 167, 169, 171, 176, 178, 190, 196, 198, 199, 200, 201, 202, 212, 214, 215, 218, 221, 223, 229, 234, 235, 240, 241, 246, 247, 248, 253, 254, 255, 256, 257, 260, 262, 268, 270, 274, 275, 276, 280, 283, 287, 290, 291, 292, 295, 303, 312, 313, 317, 326, 339, 342, 351, 353, 355, 365, 373, 374, 376, 377, 379, 390, 395, 401, 406, 408, 414, 416, 418, 421, 422, 423, 425, 426, 428, 433, 434, 437, 444, 446, 452, 454, 455, 456, 460, 470, 483, 486, 490]
con
0
AC
0.856995432066
TE
[ 69.30726509]
obj_val
69.4018528461
[4, 9, 19, 20, 48, 54, 55, 82, 83, 90, 94, 95, 98, 100, 106, 109, 111, 115, 122, 124, 126, 133, 137, 138, 139, 140, 142, 143, 147, 148, 151, 156, 157, 163, 166, 167, 169, 171, 176, 178, 190, 196, 198, 199, 200, 201, 202, 212, 214, 215, 218, 221, 223, 229, 234, 235, 240, 241, 246, 247, 248, 253, 254, 255, 256, 257, 260, 262, 268, 270,

11
AC
0.858160850364
TE
[ 31.6486316]
obj_val
31.7066810799
[4, 9, 19, 20, 48, 54, 55, 82, 83, 90, 94, 95, 98, 100, 106, 109, 111, 114, 122, 124, 126, 137, 138, 139, 140, 142, 143, 147, 148, 151, 154, 156, 157, 163, 166, 167, 169, 171, 176, 178, 180, 190, 196, 198, 199, 200, 201, 202, 212, 214, 221, 223, 229, 234, 235, 240, 241, 246, 247, 248, 254, 255, 256, 257, 260, 262, 268, 270, 275, 276, 280, 283, 287, 290, 291, 295, 303, 307, 312, 313, 317, 331, 339, 347, 351, 353, 355, 373, 374, 376, 377, 379, 395, 401, 406, 408, 411, 416, 418, 421, 422, 423, 425, 426, 428, 433, 434, 437, 442, 444, 446, 447, 452, 454, 455, 456, 460, 462, 470, 473, 483, 486, 490]
candi
cplex_val
-548.8462118446939
12
AC
0.856826929203
TE
[ 30.2519899]
obj_val
30.314634334
[4, 9, 19, 20, 48, 54, 55, 82, 83, 86, 90, 94, 95, 98, 100, 106, 111, 114, 122, 124, 126, 137, 138, 139, 140, 142, 143, 147, 148, 151, 154, 156, 157, 163, 166, 167, 169, 171, 176, 178, 180, 190, 196, 198, 199, 200, 201, 202, 212, 214, 221, 223, 

Evaluation Criteria

In [4]:
def w_i_pre_sum(w_dic):
   
    w_pre = 0  
    
    for i in w_dic.keys():


        w_pre += w_dic[i]*(dic_sedol_as[i]+1)
     
        
    w_i_pre_dic = {}

    for i in w_dic.keys():
        w_i_pre_dic.update({i:(float(w_dic[i])*(dic_sedol_as[i]+1))/w_pre})
#         print((float(w_dic[i])*(dic_sedol_as[i]+1))/w_pre)


    
    return w_i_pre_dic


def turnover(w_dic,w_dic2):
    pre_list = set(list(w_dic.keys()))

    rotp_t0 = 0

    for i in risk_sedol:
        rotp_t0 += w_dic[i]*dic_sedol_as[i]
        
        
    w_i_pre = {}
    
    w_pre = 0
    
    
    for i in risk_sedol:
        w_pre += w_dic[i]*(dic_sedol_as[i]+1)

        
        
    w_i_pre_sum = 0

    for i in risk_sedol:
        w_i_pre.update({i:(w_dic[i]*(dic_sedol_as[i]+1))/w_pre})
        w_i_pre_sum += (w_dic[i]*(dic_sedol_as[i]+1))/w_pre

        

   
        
        
    pre_list2 = set(list(w_dic2.keys()))

    turnover = 0

    add_key_1 = pre_list-pre_list2
    add_key_2 = pre_list2-pre_list


    for i in add_key_2:
        w_dic.update({i:0})

    for i in add_key_1:
        w_dic2.update({i:0})

    pre_list.union(pre_list2)

    for i in pre_list:
        turnover += abs(w_dic2[i]-w_i_pre[i])

    rOTP = 0

    for i in dic_sedol_as.keys():
        rOTP += w_dic2[i]*dic_sedol_as2[i]

    r_Txadj_t = rOTP - turnover*0.5
    
    result=[]
    result.append(turnover,rOTP,r_Txadj_t)
    result.append(w_i_pre_sum)
    
    print("turnover : " + str(turnover))
    print("r_OTP : " + str(rOTP))
    print("r_Txadj_t : " + str(r_Txadj_t))
    
    return result


def IR_TE(result,bench_t):
    cal1 = 1
    cal2 = 1
    list_std=np.array([])

    for i in range(len(result)):
        cal1 = cal1*(1+result[i])
        cal2 = cal2*(1+bench_t[i])
        list_std = np.append(list_std,resultp[i]-bench_t[i])
    
    IR = (cal1-cal2)/np.std(list_std)
    
    TE = math.sqrt(13)*np.std(list_std)
    
    result = []
    
    result.append(IR,TE)
    
    return result

def R_SR(result,bench,n):
    R_CUM = 1
    B_CUM = 1
    for i in range(len(result)):
        R_CUM = R_CUM*(1+result[t])
        
    for i in range(len(bench)):
        B_CUM = B_CUM*(1+bench[t])
    
    R_CUM = R_CUM-1
    
    B_CUM = B_CUM-1
    
    R_ANN = (1+R_CUM)**(13/n) - 1
    
    B_ANN = (1+B_CUM)**(13/n) - 1
    
    R_EXAN = R_ANN-B_ANN
    
    r_t = np.array(result)
    
    SR = R_CUM/np.std(r_t)
    
    

ipy parallel 적용

In [17]:
import os
import ipyparallel as ipp
%matplotlib

rc = ipp.Client()
# print(rc.ids)
dv = rc[:]
# dv.block=False

Using matplotlib backend: MacOSX


In [18]:
# with dv.sync_imports():
#     import class_informs as inform
    
dv.execute('import class_informs as inform')
dv.execute('ex = inform.informs(10000)')
dv.execute('ex.set_omega(risk_mat)')

# os.getcwd()

<AsyncResult: execute>

In [25]:
def cp(con) :
    new_list2 = []
    ex.set_con(cons=con)
    print("aaaaaaaaaaaaaaaaaaa")
#     try :
    aabbaa = 1
    aabbaa = ex.solve(0.01)
    if aabbaa != 0:
        aaaa = aabbaa[0]
        bbbb= aabbaa[1]
        for k in risk_sedol :
            new_list2.append(aaaa[k])

        result_list.append(aabbaa[2])
        return aabbaa[2]
    else:
        print("TE infeasible")
        return aabbaa
#     except:
#         print("no solution")
#         return 1
    return 2


def update(sol,lim_time,as_) :
    start_time = time.time()

    new_list = []          
    for k in risk_sedol :
        new_list.append(sol[k])

    result_list = []

    while(True):

        if time.time() - start_time > lim_time:
            break
        con = select_candi(new_list)
        con11 = [con]*4
        print("work???")
        print(dv)
        axz = dv.map(cp, con11)
#         print(list(axz))
        print("work!!!")
#         print(axz)
        axz.wait()
        print(axz.get())
#         print(list(axz))
        
#         adsadsadadasdasd
        axz.wait()

    return result_list

print("complete")

complete


In [26]:
#for i in date: 이런식으로 하되 evaluation criteria 는 아직 반영안함..!


############################################
date=date_list[0]
init_set='cpex' #initial set by GAN, cplex, random

c_time = 10. #cplex time for initial set
gan_rot = 2000 #GAN epoch size for initial set
############################################


#preprocessing
using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r=preprocessing(date)


if init_set=='random':
    r=list(set(list(np.random.randint(len(risk_sedol),size=70))))
    sol={}
    for i in range(len(risk_sedol)):
        if i in r: sol[risk_sedol[i]]=1.
        else: sol[risk_sedol[i]]=0.



# solve & update

as_ = using_dic['as']
result = update(sol,10.,as_)

print("------objective value 변화------")
print(result)

ok
work???
<DirectView [0, 1, 2, 3,...]>
work!!!
[0, 0, 0, 0]
ok
work???
<DirectView [0, 1, 2, 3,...]>
work!!!
[0, 0, 0, 0]
ok
work???
<DirectView [0, 1, 2, 3,...]>
work!!!
[0, 0, 0, 0]
ok
work???
<DirectView [0, 1, 2, 3,...]>
work!!!
[0, 0, 0, 0]
ok
work???
<DirectView [0, 1, 2, 3,...]>
work!!!
[0, 0, 0, 0]
ok
work???
<DirectView [0, 1, 2, 3,...]>
work!!!
[0, 0, 0, 0]
ok
work???
<DirectView [0, 1, 2, 3,...]>
work!!!
[0, 0, 0, 0]
ok
work???
<DirectView [0, 1, 2, 3,...]>
work!!!
[0, 0, 0, 0]
ok
work???
<DirectView [0, 1, 2, 3,...]>
work!!!
[0, 0, 0, 0]
ok
work???
<DirectView [0, 1, 2, 3,...]>
work!!!
[0, 0, 0, 0]
ok
work???
<DirectView [0, 1, 2, 3,...]>
work!!!
[0, 0, 0, 0]
ok
work???
<DirectView [0, 1, 2, 3,...]>
work!!!
[0, 0, 0, 0]
ok
work???
<DirectView [0, 1, 2, 3,...]>
work!!!
[0, 0, 0, 0]
ok
work???
<DirectView [0, 1, 2, 3,...]>
work!!!
[0, 0, 0, 0]
ok
work???
<DirectView [0, 1, 2, 3,...]>
work!!!
[0, 0, 0, 0]
ok
work???
<DirectView [0, 1, 2, 3,...]>
work!!!
[0, 0, 0, 0]
ok
work?